# SETUP

## Check environment

In [1]:
USE_GPU = True

## Libraries

In [2]:
# System
import os
import glob
import shutil
import copy
import re
from datetime import datetime
import logging
import sys

# Data
import numpy as np
import pandas as pd

# Visualization
import seaborn as sns
%matplotlib ipympl
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# Data processing
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import joblib

# Model
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_percentage_error
from scipy.stats import pearsonr
from keras import Input, Model, Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout, GRU, Conv1D, MaxPooling1D, Flatten
from keras.utils import plot_model
from keras.saving import load_model
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.losses import MeanAbsoluteError
from keras.losses import MeanAbsoluteError, MeanSquaredError
import keras.backend as K
from tqdm.keras import TqdmCallback
from tensorflow.keras import optimizers

# Append the custom libraries to system path
sys.path.append("/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project")

# Configure device
if not USE_GPU:
    print("Using CPU to train")
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
else:
    print("Using GPU to train")

# Custom libraries written by myself
from src.plot import plot_1_data, plot_2_data, plot_3_data, plot_prediction, plot_learning_curves
from src.reduction_model.lstm_s2s import LSTMSeq2SeqReduction
from src.reduction_model.gru_s2s import GRUSeq2SeqReduction
from src.reduction_model.cnnlstm_s2s import CNNLSTMSeq2SeqReduction
from src.prediction_model.lstm import LSTMPrediction, inferenceLSTM
from src.loop_model import generate_loopresults, choose_the_best
from src.reduce_data_utils import prepareReducedData
from src.data_utils import mice
from src.time_series_utils import splitTrainValidationTestTimeSeries, reframePastFuture, padPastFuture

# Configuration reader
from src.config_reader import ConfigurationReader

# Checking Tensorflow
print(f"Tensorflow version: {tf.__version__}, GPUs: {tf.config.list_physical_devices('GPU')}")

E0000 00:00:1762136767.950593     280 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762136767.977862     280 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762136768.257738     280 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762136768.257785     280 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762136768.257787     280 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762136768.257789     280 computation_placer.cc:177] computation placer already registered. Please check linka

Using GPU to train
Tensorflow version: 2.19.1, GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Configuration

In [3]:
conf = ConfigurationReader("/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/model_params.json").data
conf

{'dataset': {'aod2022': {'file_dir': '/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/MatchingData2022.xlsx',
   'target_start_date': '2022-01-01',
   'target_end_date': '2022-12-31'},
  'aod2021': {'file_dir': '/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/aod_data_daily.csv',
   'target_start_date': '2021-01-01',
   'target_end_date': '2021-12-31'},
  'mpair': {'file_dir': '/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/MPair.csv',
   'target_start_date': '2021-01-01',
   'target_end_date': '2022-12-31',
   'station_2022_dir': '/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/station2022.csv',
   'station_2018_2021_dir': '/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/station2018-2021.csv',
   'merged_data_dir': '/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/mpair_merged.csv',
   'merged_data_dir_all_locatio

# DATA FROM VIET

## Load dataset

In [4]:
dataset_path = "/le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station"

df_raw = []

for csv_file in sorted(glob.glob(f"{dataset_path}/*")):
    print(f"Working on {csv_file}")
    df_station = pd.read_csv(csv_file)
    df_station["station"] = int(os.path.splitext(os.path.basename(csv_file))[0])
    df_raw.append(df_station)
df_raw = pd.concat(df_raw, axis=0)

# Lowercase columns name
df_raw.columns = df_raw.columns.map(lambda col: col.lower().strip())

# Set time index
df_raw["date"] = pd.to_datetime(df_raw["date"])
df_raw.set_index("date", inplace=True)

# Drop columns
features_to_use = ["pm25_quantrac", "tsp_quantrac", "station"]
#cols_to_drop = list(filter(lambda col: "vientham" in col or "cmaq" in col, df_raw.columns)) + ["unnamed: 0"]
#print(f"Columns to drop = {cols_to_drop}")
#df_raw = df_raw.drop(columns=cols_to_drop)
df_raw = df_raw.loc[:, features_to_use]

df_raw

Working on /le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station/211.csv
Working on /le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station/212.csv
Working on /le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station/213.csv
Working on /le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station/214.csv
Working on /le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station/215.csv
Working on /le_thanh_van_118/workspace/hiep_workspace/air_quality_index_project/dataset/merged-data/by-station/216.csv


KeyError: 'sit'

In [ ]:
stop

## Data preparation

### Drop missing

In [ ]:
df = []
for station in sorted(df_raw["station"].unique()):
    df_station = df_raw[df_raw["station"] == station]
    df.append(df_station.dropna())
df = pd.concat(df, axis=0)
df

### Scale

In [ ]:
X = df.loc[:, ["tsp_quantrac"]]
y = df.loc[:, ["pm25_quantrac"]]

feature_scaler = MinMaxScaler()
X_scaled = feature_scaler.fit_transform(X)
print(X_scaled)

label_scaler = MinMaxScaler()
y_scaled = label_scaler.fit_transform(y)
print(y_scaled)

df_scaled = pd.DataFrame(np.concatenate((X_scaled, y_scaled), axis=1), index=df.index)
df_scaled["station"] = df.loc[:, ["station"]]
df_scaled.columns = list(X.columns) + list(y.columns) + ["station"]

df_scaled

### Train test split

In [ ]:
df_scaled = df_scaled.drop(columns=["station"], errors="ignore")

splitter = round(0.8 * len(df_scaled))
train_data = df_scaled.iloc[:splitter]
test_data = df_scaled.iloc[splitter:]

display(train_data)
display(test_data)

## Prediction

### Model paramaters

In [ ]:
n_past = 168
n_future = 4
n_features = 2
n_label = 1
latent_dim = 32

epochs = 200
batch_size = 128

In [ ]:
# Calculate MNBE
def mean_normalized_bias_error(y_pred, y_actual):
    y_pred = np.array(y_pred)
    y_actual = np.array(y_actual)
    return np.mean((y_pred - y_actual) / np.mean(y_actual)) * 100

metrics_calculators = {
    "mae": mean_absolute_error,
    "mse": mean_squared_error,
    "rmse": root_mean_squared_error,
    "r2": r2_score,
    "mape": mean_absolute_percentage_error,
    "mnbe": mean_normalized_bias_error,
    "r_coeff": lambda a,b: pearsonr(a, b)[0]
}

### Reframe data

In [ ]:
# Function to reframe data
def reframe(data, n_past, n_future, n_shifted=0):
    m = np.arange(0, len(data) - n_past - n_future - n_shifted + 1)
    X = np.array([data.iloc[i:(i + n_past), :] for i in m])
    y = np.expand_dims(np.array([data.iloc[(i + n_shifted + n_past):(i + n_shifted + n_past + n_future), -1] for i in m]), -1)
    return X, y

X_train, y_train = reframe(train_data, n_past, n_future)
X_test, y_test = reframe(test_data, n_past, n_future)

X_train_shifted_1, y_train_shifted_1 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_1, y_test_shifted_1 = reframe(test_data, n_past, n_future, 1)
print(X_train_shifted_1.shape, y_train_shifted_1.shape, X_test_shifted_1.shape, y_test_shifted_1.shape)

X_train_shifted_2, y_train_shifted_2 = reframe(train_data, n_past, n_future, 2)
X_test_shifted_2, y_test_shifted_2 = reframe(test_data, n_past, n_future, 2)
print(X_train_shifted_2.shape, y_train_shifted_2.shape, X_test_shifted_2.shape, y_test_shifted_2.shape)

X_train_shifted_3, y_train_shifted_3 = reframe(train_data, n_past, n_future, 3)
X_test_shifted_3, y_test_shifted_3 = reframe(test_data, n_past, n_future, 3)
print(X_train_shifted_3.shape, y_train_shifted_3.shape, X_test_shifted_3.shape, y_test_shifted_3.shape)

X_train_shifted_4, y_train_shifted_4 = reframe(train_data, n_past, n_future, 4)
X_test_shifted_4, y_test_shifted_4 = reframe(test_data, n_past, n_future, 4)
print(X_train_shifted_4.shape, y_train_shifted_4.shape, X_test_shifted_4.shape, y_test_shifted_4.shape)

### LSTM-Seq2Seq

In [ ]:
def define_lstms2s_model(n_past, n_future, n_features, n_label, name="model"):
    # Encoder layers
    encoder_inputs = Input(shape=(n_past, n_features))
    encoder_lstm_1 = LSTM(128, return_sequences=True, activation="relu")(encoder_inputs)
    encoder_lstm_2, state_h, state_c = LSTM(64, return_state=True, activation="relu", dropout=0.2)(encoder_lstm_1)
    encoder_dense = Dense(latent_dim)(encoder_lstm_2)
    # Repeat layer
    decoder_repeat_vector = RepeatVector(n_future)(encoder_dense)
    # Decoder layers
    decoder_lstm_1 = LSTM(64, return_sequences=True, activation="relu")(decoder_repeat_vector, initial_state=[state_h, state_c])
    decoder_lstm_2 = LSTM(128, return_sequences=True, activation="relu", dropout=0.2)(decoder_lstm_1)
    decoder_outputs = TimeDistributed(Dense(n_label))(decoder_lstm_2)

    # Compile the model
    model = Model(encoder_inputs, decoder_outputs)
    model.name = name
    model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())
    display(model.summary())
    return model

#### No shift

In [ ]:
lstms2s = define_lstms2s_model(n_past, n_future, n_features, n_label, "lstms2s_no_shift")

history = lstms2s.fit(X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred = lstms2s.predict(X_test)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 1

In [ ]:
X_train_shifted_1, y_train_shifted_1 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_1, y_test_shifted_1 = reframe(test_data, n_past, n_future, 1)

lstms2s_shifted1 = define_lstms2s_model(n_past, n_future, n_features, n_label, "lstms2s_shifted_1")

history = lstms2s_shifted1.fit(X_train_shifted_1, y_train_shifted_1,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_1 = lstms2s_shifted1.predict(X_test_shifted_1)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_1[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_1[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 2

In [ ]:
X_train_shifted_2, y_train_shifted_2 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_2, y_test_shifted_2 = reframe(test_data, n_past, n_future, 1)

lstms2s_shifted_2 = define_lstms2s_model(n_past, n_future, n_features, n_label, "lstms2s_shifted_2")

history = lstms2s_shifted_2.fit(X_train_shifted_2, y_train_shifted_2,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_2 = lstms2s_shifted_2.predict(X_test_shifted_2)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_2[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_2[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 3

In [ ]:
X_train_shifted_3, y_train_shifted_3 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_3, y_test_shifted_3 = reframe(test_data, n_past, n_future, 1)

lstms2s_shifted_3 = define_lstms2s_model(n_past, n_future, n_features, n_label, "lstms2s_shifted_3")

history = lstms2s_shifted_3.fit(X_train_shifted_3, y_train_shifted_3,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_3 = lstms2s_shifted_3.predict(X_test_shifted_3)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_3[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_3[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 4

In [ ]:
lstms2s_shifted4 = define_lstms2s_model(n_past, n_future, n_features, n_label, "lstms2s_shifted_4")

history = lstms2s_shifted4.fit(X_train_shifted_4, y_train_shifted_4,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_4 = lstms2s_shifted4.predict(X_test_shifted_4)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_4[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_4[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

### GRU-Seq2Seq

In [ ]:
def define_grus2s_model(n_past, n_future, n_features, n_label, name="model"):
    # encoder layers
    encoder_inputs = Input(shape=(n_past, n_features))
    encoder_gru_1 = GRU(128, return_sequences=True, activation="relu")(encoder_inputs)
    encoder_gru_2, state_h = GRU(64, return_state=True, activation="relu")(encoder_gru_1)
    encoder_dropout = Dropout(0.2)(encoder_gru_2)
    encoder_dense = Dense(latent_dim)(encoder_dropout)
    # Repeat layer
    decoder_repeat_vector = RepeatVector(n_future)(encoder_dense)
    # Decoder layers
    decoder_gru_1 = GRU(64, return_sequences=True, activation="relu")(decoder_repeat_vector, initial_state=state_h)
    decoder_gru_2 = GRU(128, return_sequences=True, activation="relu")(decoder_gru_1)
    decoder_dropout = Dropout(0.2)(decoder_gru_2)
    decoder_outputs = TimeDistributed(Dense(n_label))(decoder_dropout)
    # Compile the model
    model = Model(encoder_inputs, decoder_outputs)
    model.name = name
    model.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())
    display(model.summary())
    return model

#### No shift

In [ ]:
grus2s = define_grus2s_model(n_past, n_future, n_features, n_label, "grus2s_no_shift")

history = grus2s.fit(X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred = grus2s.predict(X_test)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 1

In [ ]:
X_train_shifted_1, y_train_shifted_1 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_1, y_test_shifted_1 = reframe(test_data, n_past, n_future, 1)

grus2s_shifted1 = define_grus2s_model(n_past, n_future, n_features, n_label, "grus2s_shifted_1")

history = grus2s_shifted1.fit(X_train_shifted_1, y_train_shifted_1,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_1 = grus2s_shifted1.predict(X_test_shifted_1)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_1[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_1[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 2

In [ ]:
X_train_shifted_2, y_train_shifted_2 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_2, y_test_shifted_2 = reframe(test_data, n_past, n_future, 1)

grus2s_shifted_2 = define_grus2s_model(n_past, n_future, n_features, n_label, "grus2s_shifted_2")

history = grus2s_shifted_2.fit(X_train_shifted_2, y_train_shifted_2,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_2 = grus2s_shifted_2.predict(X_test_shifted_2)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_2[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_2[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 3

In [ ]:
X_train_shifted_3, y_train_shifted_3 = reframe(train_data, n_past, n_future, 1)
X_test_shifted_3, y_test_shifted_3 = reframe(test_data, n_past, n_future, 1)

grus2s_shifted_3 = define_grus2s_model(n_past, n_future, n_features, n_label, "grus2s_shifted_3")

history = grus2s_shifted_3.fit(X_train_shifted_3, y_train_shifted_3,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_3 = grus2s_shifted_3.predict(X_test_shifted_3)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_3[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_3[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

#### Shifted 4

In [ ]:
grus2s_shifted4 = define_grus2s_model(n_past, n_future, n_features, n_label, "grus2s_shifted_4")

history = grus2s_shifted4.fit(X_train_shifted_4, y_train_shifted_4,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred_shifted_4 = grus2s_shifted4.predict(X_test_shifted_4)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test_shifted_4[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred_shifted_4[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

### CNN-LSTM Seq2seq

In [ ]:
# Encoder layers
encoder_inputs = Input(shape=(n_past, n_features))
encoder_cnn_1 = Conv1D(filters=128, kernel_size=2, activation="relu")(encoder_inputs)
encoder_max_pooling_1 = MaxPooling1D(pool_size=2)(encoder_cnn_1)
encoder_cnn_2 = Conv1D(filters=64, kernel_size=2, activation="relu")(encoder_max_pooling_1)
encoder_max_pooling_2 = MaxPooling1D(pool_size=2)(encoder_cnn_2)
encoder_lstm, state_h, state_c = LSTM(64, return_state=True, activation="relu")(encoder_max_pooling_2)
encoder_dropout = Dropout(0.2)(encoder_lstm)
encoder_dense = Dense(latent_dim)(encoder_dropout)
# Repeat layer
decoder_repeat_vector = RepeatVector(n_future)(encoder_dense)
# Decoder layers
decoder_lstm_1 = LSTM(64, return_sequences=True, activation="relu")(decoder_repeat_vector, initial_state=[state_h, state_c])
decoder_dropout = Dropout(0.2)(decoder_lstm_1)
decoder_outputs = TimeDistributed(Dense(n_label))(decoder_dropout)
# Compile the model
cnncnnlstms2smodel = Model(encoder_inputs, decoder_outputs)
# Compile the model
cnnlstms2smodel = Model(encoder_inputs, decoder_outputs)
cnnlstms2smodel.compile(optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError())
display(cnnlstms2smodel.summary())

# Fit model
history = cnnlstms2smodel.fit(X_train, y_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_split=0.2,
                shuffle=False,
                callbacks = [
                    #EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, restore_best_weights=True),
                    TqdmCallback(verbose=1)
                ],
                verbose=1)
plot_learning_curves(history)

# Predict data
y_pred = cnnlstms2smodel.predict(X_test)

# Evaluation
for day in range(n_future):
    print(f"Day = {day}")
    inv_y_test = label_scaler.inverse_transform(y_test[:, day, :])[:-(day+1)]
    inv_y_pred = label_scaler.inverse_transform(y_pred[:, day, :])[(day+1):]
    for metric, calculator in metrics_calculators.items():
        print(f"{metric}: {calculator(inv_y_pred, inv_y_test)}")
    plot_2_data(data1=inv_y_test, datalabel1="y_test", data2=inv_y_pred, datalabel2="y_predicted")

## Evaluation board

In [ ]:
pd.DataFrame(evaluation_board)